In [36]:
import os
import operator
import csv
import math
from collections import OrderedDict

In [37]:
temp_topics=[]
ques_groups={}
ans_groups={}
topic_dict={}

In [38]:
def reading_topics():
    topics = []
    count=0
    with open('item_id_q_tags.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            topics.append(row[1])
            if row[1] in topic_dict:
                t=1
            else:
                topic_dict[row[1]]=count
                count += 1
    return topics

In [39]:
def reading_data(topics):
    test_data = {}
    for filename in os.listdir('./kahi_unkahi_ques'):
        filenumber = int(filename[4:-4])
        filename = './kahi_unkahi_ques/' + filename
        doc = open(filename,encoding='UTF-8').read()
        doc = doc[:-1]
        topic = topics[filenumber]
        if(topic in ques_groups):
            ques_groups[topic].append(filenumber)
        else:
            temp = []
            ques_groups[topic]=temp
            ques_groups[topic].append(filenumber)
        if topic in test_data:
            test_data[topic] = test_data[topic] + " " + doc	
        else:
            test_data[topic] = doc
    count = 0
    c = 0
    for filename in os.listdir('./kahi_unkahi_ans'):
        filenumber = int(filename[4:-4])
        filename = './kahi_unkahi_ans/' + filename
        doc = open(filename,encoding='UTF-8').read()
        #doc = doc.decode("utf-8")
        if(doc == ""):
            continue
        c+=1
        words = doc.splitlines()
        words = words[3:]		
        output = ""
        for word in words:
            temp = word.split()
            count = 0
            for a in temp:
                if(count == 2):
                    output += a
                    output += " "
                if(a == '-'):
                    count = count+1
            if(output==""):continue
        topic = topics[filenumber]
        if(topic in ans_groups):
            ans_groups[topic].append(filenumber)
        else:
            temp = []
            ans_groups[topic]=temp
            ans_groups[topic].append(filenumber)
    data = []
    for key in test_data.keys():
        #print(key)
        if(key == ""):
            print(test_data[key])
        temp_topics.append(key)
        data.append(test_data[key])
    return data

In [40]:
def cleaning(data):
    for i in range(0, len(data)):
        temp=""
        for j in range(0, len(data[i])):
            if(data[i][j] != ',' and data[i][j] != ')' and data[i][j] != '(' and data[i][j] != '"' and data[i][j] != ':' and data[i][j] != '%' and not(data[i][j] >= '0' and data[i][j] <= '9') and data[i][j] != '-' and data[i][j] != '/' and not(data[i][j] >= 'a' and data[i][j] <= 'z') and not(data[i][j] >= 'A'  and data[i][j] <= 'Z') and data[i][j] != '/' and data[i][j] != '-'):
                temp=temp+data[i][j]
        data[i]=temp
    new_data=data
    for i in range(0, len(new_data)):
        if(i == 0):
            continue
        temp = new_data[i].split()
        le = len(temp)
        if(le !=0 and temp[le - 1] == '?'):
            out = ""
            for j in range(le-1):
                out += temp[j]
                out += " "
            new_data[i] = out
    return new_data

In [41]:
def stopwords_removal(data):
    stop_file=open("stopwords-hi.txt",encoding='UTF-8').read()
    #stop_file=stop_file.decode("utf-8")
    stopwords=stop_file.split()
    dictionary={}
    for word in stopwords:
        dictionary[word]=1
    for i in  range(len(data)):
        temp=""
        arr=data[i].split()
        for j in range(len(arr)):
            if(arr[j] not in dictionary):temp+=(arr[j] + " ")
        data[i]=temp
    return data

In [42]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, counts in wordDict.items():
        tfDict[word] = counts/float(bowCount)
    return tfDict

In [43]:
def makewordDict(data):
    dictionary = {}
    line = data.split()
    for word in line:
        if word in dictionary:
            dictionary[word] = dictionary[word] + 1
        else:
            dictionary[word] = 1
    return dictionary

In [44]:
def makeTF(data):
    answer = []
    for bow in data:
        dictionary = makewordDict(bow)
        tfDict = computeTF(dictionary, bow)
        answer.append(tfDict)
    print(len(data))
    return answer

In [45]:
def computeIDF(data):
    idfDict = {}
    for bow in data:
        line = bow.split()
        my_set = set(line)
        for word in my_set:
            if word in idfDict:
                idfDict[word] += 1
            else:
                idfDict[word] = 1
    for word, val in idfDict.items():
        idfDict[word] = math.log10(len(data) / float(val)) 
    return idfDict

In [46]:
def computeTFIDF(tfbow, idfs):
    tfidf = {}
    for word, val in tfbow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [47]:
def makeTFIDF(tfdata, idfdata):
    answer = []
    for tfbow in tfdata:
        answer.append(computeTFIDF(tfbow, idfdata))
    print (len(answer))
    print (len(tfdata))
    return answer

In [52]:
def accuracymeasure():
    count = 0
    c = 0
    test_data = []
    test_data_filenumber = []
    for filename in os.listdir('./kahi_unkahi_ans'):
        filenumber = int(filename[4:-4])
        filename = './kahi_unkahi_ans/' + filename
        doc = open(filename,encoding='UTF-8').read()
        #doc = doc.decode("utf-8")
        if(doc == ""):
            continue
        c+=1
        words = doc.splitlines()
        words = words[3:]
        output = ""
        for word in words:
            temp = word.split()
            count = 0
            for a in temp:
                if(count == 2):
                    output += a
                    output += " "
                if(a == '-'):
                    count = count+1
            if(output==""):continue
        topic = topics[filenumber]
        test_data_filenumber.append(filenumber)
        test_data.append(output)
    test_data=cleaning(test_data)

    test_data=stopwords_removal(test_data)
    split_test_data=[]
    for i in range(0, len(test_data)):
        temp=test_data[i]
        temp=temp.split()
        split_test_data.append(temp)
    correct=0
    total=0
#     print (len(split_test_data))
    
    prediction_matrix = []
    real_matrix = []
    for i in range(0,len(split_test_data)):
        temp=[]
        for j in range(0, len(topic_dict)):
            temp.append(0)
        prediction_matrix.append(temp)
        temp=[]
        for j in range(0, len(topic_dict)):
            temp.append(0)
        real_matrix.append(temp)
        max_score=0.0
        max_index=-1
        score_table = {}
        for j in range(1, len(temp_topics)-1):
            score_table[temp_topics[j]] = 0.0
        for j in range(0, len(sorted_tfidfdata)):
            tfidf=sorted_tfidfdata[j]
            curr_score=0.0
            for k in range(0, len(split_test_data[i])):
                word=split_test_data[i][k]
                for word1, val1 in tfidf.items():
                    if(word == word1):
                        curr_score=curr_score+val1
                        break
            sum1=0.0
            for word1, val1 in tfidf.items():
                sum1=sum1+val1
            if(sum1==0):
                sum1=1
            curr_score = curr_score/sum1
            score_table[temp_topics[j+1]]=curr_score
            if(curr_score > max_score):
                max_score=curr_score
                max_index=j
        test_data_topic=temp_topics[max_index+1]
        real_data_topic=topics[test_data_filenumber[i]]
        test_data_topic_index=topic_dict[test_data_topic]
        real_data_topic_index=topic_dict[real_data_topic]
        prediction_matrix[i][test_data_topic_index]=1
        real_matrix[i][real_data_topic_index]=1
        to_find = test_data_filenumber[i]
        check=0
        for j in range(0, len(ques_groups[test_data_topic])):
            if(to_find == ques_groups[test_data_topic][j]):
                check=1
                break
        if(check == 1):
            correct = correct + 1
            # print "matched"
        totall = total + 1
#     print(correct)
#     print(total)
    print (correct/len(test_data))
    global_true_positives=1
    global_false_positives=1
    global_true_negatives=1
    global_false_negatives=1
    for topic in topic_dict.keys():
        topic_index=topic_dict[topic]
        true_positives=1
        false_positives=1
        true_negatives=1
        false_negatives=1
        for i in range(0, len(prediction_matrix)):
            if(prediction_matrix[i][topic_index] == 1):
                if(prediction_matrix[i][topic_index] == real_matrix[i][topic_index]):
                    true_positives +=1
                    global_true_positives += 1
                else:
                    false_positives += 1
                    global_false_positives += 1
            else:
                if(prediction_matrix[i][topic_index] == real_matrix[i][topic_index]):
                    true_negatives += 1
                    global_true_negatives += 1
                else:
                    false_negatives += 1
                    global_false_negatives += 1
        accuracy = (true_positives + true_negatives)/(true_positives + true_negatives + false_positives + false_negatives)
        precision = true_positives/(true_positives + false_positives)
        recall = true_positives/(true_positives + false_negatives)
        f1_score = 2/((1/precision) + (1/recall))
        print( "Topic : ", topic)
        print ("ACcuracy : ", accuracy*100.0)
        print ("Precision : ", precision*100.0)
        print ("Recall : ", recall*100.0)
        print ("F1_score : ", f1_score*100.0)
    accuracy = (global_true_positives + global_true_negatives)/(global_true_positives + global_true_negatives + global_false_positives + global_false_negatives)
    precision = global_true_positives/(global_true_positives + global_false_positives)
    recall = global_true_positives/(global_true_positives + global_false_negatives)
    f1_score = 2/((1/precision) + (1/recall))
    print( "ACcuracy : ", accuracy*100.0)
    print ("Precision : ", precision*100.0)
    print ("Recall : ", recall*100.0)
    print( "F1_score : ", f1_score*100.0)


In [53]:
topics = reading_topics()
data = reading_data(topics)
data = cleaning(data)

# print (len(data))
data = stopwords_removal(data)

tfdata = makeTF(data)
idfdata = computeIDF(data)
tfidfdata = makeTFIDF(tfdata, idfdata)
sorted_tfidfdata = []
for tfidf in tfidfdata:
    sorted_tfidf = OrderedDict(sorted(tfidf.items(), key=operator.itemgetter(1), reverse=True))
    sorted_tfidfdata.append(sorted_tfidf)

count = 0

# print (len(temp_topics))
# print (len(sorted_tfidfdata))
accuracymeasure()

# for tfidf in sorted_tfidfdata:
#     filename="./temp/" + temp_topics[count] + ".txt"
#     file=open(filename, 'w')
#     # print temp_topics[count]
#     count = count + 1
#     for word, val in tfidf.items():
#         file.write(word + " " + str(val))
#         file.write('\n')

  
38
38
38
0.012987012987012988
Topic :  First period
ACcuracy :  95.64516129032258
Precision :  4.545454545454546
Recall :  14.285714285714285
F1_score :  6.896551724137931
Topic :  Irregular periods
ACcuracy :  98.06451612903226
Precision :  28.57142857142857
Recall :  22.22222222222222
F1_score :  25.0
Topic :  Other
ACcuracy :  93.54838709677419
Precision :  50.0
Recall :  2.5
F1_score :  4.761904761904762
Topic :  STDs
ACcuracy :  98.38709677419355
Precision :  33.33333333333333
Recall :  11.11111111111111
F1_score :  16.666666666666664
Topic :  Masturbation
ACcuracy :  93.2258064516129
Precision :  22.22222222222222
Recall :  5.405405405405405
F1_score :  8.695652173913043
Topic :  Pregnancy
ACcuracy :  93.2258064516129
Precision :  11.11111111111111
Recall :  2.857142857142857
F1_score :  4.545454545454546
Topic :  Sex
ACcuracy :  56.12903225806451
Precision :  25.0
Recall :  0.7462686567164178
F1_score :  1.4492753623188406
Topic :  Other sexual health issues
ACcuracy :  86.45